### Testbed for constructing Binance Market Module
1. Create module to ingest data as flat files
2. Allow for pagination to resolve for API rate limit
3. Daily data upload from Binance API
4. Create module to retrieve files

In [147]:
# Binance Market Module Testbed

import numpy as np
import pandas as pd
import time
import datetime as dt
import math
import os
import requests  # requests module to interact with endpoints

In [230]:
# construct class to store exchange information and ingest data, for Spot
class BinanceMarketModule:
    
    # provide class variables
    spot_base_url = 'https://api3.binance.com'
    lin_futures_base_url = 'https://fapi.binance.com'
    inv_futures_base_url = 'https://dapi.binance.com'
    
    # store symbolmaster paths
    spot_smpath = 'spot_symbolmaster.feather'
    lin_futures_smpath = 'lin_symbolmaster.feather'
    inv_futures_smpath = 'inv_symbolmaster.feather'
    
    # intervals: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    
    # initialise constructor to token_type input
    def __init__(self, token_type: str):
        self.token_type = token_type
        self._assign_api()  # assign the right api
        self.base_url = self._assign_api()
        self.smpath = self._smpath()
        self.symbolmaster = None
        self.tokenlist = None
        
    # update symbolmaster
    def pullSymbolMaster(self) -> pd.DataFrame:
        # check if symbolmaster already exists in local directory
        self.smpath= self._smpath()
        if os.path.exists(self.smpath):
            df = pd.read_feather(self.smpath)

        # if symbolmaster does not currently exist, construct new symbolmaster
        else:
            exchange_url = '/api/v3/exchangeInfo'
            endpoint = self.base_url + exchange_url
            response = self._get(endpoint)
            symbolmaster_cols = ['symbol', 'status', 'baseAsset', 'quoteAsset']
            df = pd.DataFrame(response['symbols'])[symbolmaster_cols]
            df['timestamp_utc'] = dt.datetime.now(dt.timezone.utc)
            df['date_utc'] = df['timestamp_utc'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        
        self.symbolmaster = df
        self.tokenlist = df['symbol'].values
        return self.symbolmaster
    
    # store symbolmaster as feather
    def storeSymbolMaster(self):
        # if it currently does not exist
        if self.symbolmaster is None:
            self.pullSymbolMaster()
        # if symbolmaster exists
        else:
            self.symbolmaster.to_feather(self.smpath)
            print(f'Symbol Master stored as {self.smpath}')
    
    # single pull ticker data
    def pullOHLCV(self, symbol: str, interval: str, start: float, end: float)-> pd.DataFrame:
        # manage duration
        unix_start = self.string_to_unixtime(start)
        unix_end = self.string_to_unixtime(end)
        
        kline_url = '/api/v3/klines'
        full_url = self.base_url + kline_url
        params = {'symbol': symbol,
                  'interval': interval,
                  'limit': 1000,
                  'startTime': unix_start,
                  'endTime': unix_end
                 }
        result = self._get(full_url, params=params)
        cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'trades']
        df = pd.DataFrame(result).iloc[:, :7]
        df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time']
        df['open_ts'] = df['open_time'].apply(lambda x: dt.datetime.fromtimestamp(x/1000))
        df['close_ts'] = df['close_time'].apply(lambda x: dt.datetime.fromtimestamp(x/1000))
        df = df[['open_ts', 'open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                'close_ts']]
        return df
            
    # assign appropriate api_urls
    def _assign_api(self) -> str:
        if self.token_type == 'spot':
            self.base_url = self.spot_base_url
        elif self.token_type == 'lin_future':
            self.base_url = self.lin_futures_base_url
        elif self.token_type == 'inv_future':
            self.base_url = self.inv_futures_base_url
        else:
            self.base_url = None
            print('Unknown Token Type')
        return self.base_url
            
    # get request for usage within the class
    def _get(self, full_endpoint: str, params: dict = None):
        r = requests.get(full_endpoint, params=params)
        results = r.json()
        return results
    
    def _smpath(self) -> str:
        if self.token_type == 'spot':
            self.smpath = self.spot_smpath
        elif self.token_type == 'lin_future':
            self.smpath = self.lin_futures_smpath
        elif self.token_type == 'inv_future':
            self.smpath = self.inv_futures_smpath
        else:
            self.smpath = None
            print('Unknown Token Type')
        return self.smpath
    
    # create methods for handling time
    
    @staticmethod
    def string_to_unixtime(timestring: str):  # convert this into UTC unixtime
        unixtime = time.mktime(dt.datetime.strptime(timestring, '%Y-%m-%d %H:%M:%S').timetuple()) * 1000
        return int(unixtime)

In [231]:
b = BinanceMarketModule('spot')

In [232]:
sm = b.pullSymbolMaster()

In [233]:
b.storeSymbolMaster()

Symbol Master stored as spot_symbolmaster.feather


In [234]:
start = '2022-12-01 00:00:00'
end = '2022-12-10 00:00:00'
df = b.pullOHLCV('BTCUSDT', '1h', start, end)

In [235]:
df.tail()

,open_ts,open_time,open,high,low,close,volume,close_time,close_ts
212,2022-12-09 20:00:00,1670587200000,17241.16000000,17360.00000000,17170.85000000,17256.24000000,20501.81159000,1670590799999,2022-12-09 20:59:59.999
213,2022-12-09 21:00:00,1670590800000,17257.31000000,17275.22000000,17058.21000000,17150.70000000,24062.35206000,1670594399999,2022-12-09 21:59:59.999
214,2022-12-09 22:00:00,1670594400000,17151.55000000,17173.94000000,17079.83000000,17156.25000000,16801.33790000,1670597999999,2022-12-09 22:59:59.999
215,2022-12-09 23:00:00,1670598000000,17156.25000000,17199.36000000,17136.94000000,17181.50000000,13206.96108000,1670601599999,2022-12-09 23:59:59.999
216,2022-12-10 00:00:00,1670601600000,17181.50000000,17187.51000000,17152.07000000,17159.90000000,10109.02569000,1670605199999,2022-12-10 00:59:59.999


In [205]:
df['close_ts'] = df['close_time'].apply(lambda x: dt.datetime.fromtimestamp(x/1000))

In [206]:
df

,open_time,open,high,low,close,volume,close_time,ts
0,1670410800000,16826.09000000,16832.85000000,16780.33000000,16793.29000000,8411.21752000,1670414399999,2022-12-07 19:59:59.999
1,1670414400000,16793.29000000,16810.81000000,16765.12000000,16794.22000000,8176.75442000,1670417999999,2022-12-07 20:59:59.999
2,1670418000000,16794.21000000,16872.00000000,16793.00000000,16855.24000000,12520.82420000,1670421599999,2022-12-07 21:59:59.999
3,1670421600000,16855.24000000,16897.38000000,16821.15000000,16837.91000000,11855.93357000,1670425199999,2022-12-07 22:59:59.999
4,1670425200000,16838.86000000,16858.88000000,16810.01000000,16837.48000000,13029.04588000,1670428799999,2022-12-07 23:59:59.999
...,...,...,...,...,...,...,...,...
495,1672192800000,16695.55000000,16713.41000000,16677.42000000,16683.81000000,5590.53803000,1672196399999,2022-12-28 10:59:59.999
496,1672196400000,16683.81000000,16694.68000000,16559.79000000,16650.65000000,14186.30434000,1672199999999,2022-12-28 11:59:59.999
497,1672200000000,16650.65000000,16678.85000000,16641.25000000,16660.44000000,5941.38788000,1672203599999,2022-12-28 12:59:59.999
498,1672203600000,16660.15000000,16668.00000000,16585.71000000,16602.98000000,10723.34379000,1672207199999,2022-12-28 13:59:59.999


In [20]:
endpoint

'https://api3.binance.com/api/v3/exchangeInfo'

In [41]:
# procure symbol master
symbols = [x['symbol'] for x in response.json()['symbols'] if x['symbol'].__contains__('USDT')]

In [44]:
response.status_code

200

In [47]:
response.json()

{'timezone': 'UTC',
 'serverTime': 1672196066190,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelReplaceAllowed': True,
   'i

limit_output extension: Maximum message size of 10000 exceeded with 1770841 characters

In [45]:

# check the status code
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print('Error: status code {}'.format(response.status_code))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

